# Titanic. A learning journey

## Introduction
This is a mix that I'm learning from different Titanic noteboks. The main inspiratiors are

* [Titanic Survival Predictions (Beginner)](https://www.kaggle.com/nadintamer/titanic-survival-predictions-beginner)
* [A Data Science Framework: To Achieve 99% Accuracy](https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy)

Contents: Import Necessary Libraries Read In and Explore the Data Data Analysis Data Visualization Cleaning Data Choosing the Best Model Creating Submission File Any and all feedback is welcome!

## Table of Contents
1. [Define the problem](#ch1)
1. [Read the data](#ch2)


<a id="ch1"></a>

## 1. Define the problem

First thing first. What we are trying to do? 

Titanic. Belfast 1912. The largest ship ever made was struck by an iceberg and sank in his maiden vollage. Safety deficencies caused that more tan 1,500 of the 2,224. Our task is guess if a passenger will survive or not.

Well, let's do it 

<a id="ch2"></a>


## 2. Read the data

To start we begin reading and exploring the data 


In [ ]:
#data analysis libraries 
import numpy as np
import pandas as pd

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#import train and test CSV files
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

#take a look at the training data
print (train.info())
train.describe(include="all")


In [ ]:
# A quick lock to the content
train.head()

In [ ]:
# Where are missing values
print(pd.isnull(train).sum())
print(pd.isnull(test).sum())

In [ ]:
print(test.Parch.unique())
print(train.Parch.unique())

print(test.Embarked.unique())
print(train.Embarked.unique())

# Data recap

   Well, after doig a first look to the data. What we can say?

   * PassengerId: Id of the passenger, doesn't seems relevant
   * Survived: What we want to know (0/1)
   * Pclass: A proxy for SES(Socio-Economic status) (1/2/3). 
   * Name: String with the name. Could we extract some information from here? 
   * Sex: Descriptior of the sex (male/female)
   * Age: Age of the person
   * SibSp: Number of siblings spouses abroad  (0- 8)
   * Parch: Number of parent children abroad (0 - 9)
   * Ticket: String Id of the ticket
   * Fare: Passenger Fare
   * Cabin: Cabin used. A lot of nulls
   * Embark: Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
  
      

   First thoughts

   * PClass is a integer but could make sense to tranform to a category?
   * Name + SibSp + Parch + ticket + cabin. Could be used to identify families? 

# discover correlations
  
    First we will try to discover correlations between values and result



In [ ]:

for x in train:
    if ((x != "Survived") and (train[x].unique().size <=10)):
        print (x, ": ", train[x].unique().size)
        print (train[["Survived",x]].groupby(x, as_index=False).agg(['count', 'mean']))


Well, this is interesting for what it says and for what is missig.
Class, sex are very imoportant
also it seems that there are a correlation between size of the family (Parch, sibSp) and probabilities of supervivence
what is missing is the age, as is provided is not very useful. We will copy the way that is arranged in the notebook of LD Freeman, and by the way, it seems that he is also doing a interesting thing as is extract the Title form the name. Ok, let's do it 

In [ ]:
for dataset in data_cleaner:    
    #Discrete variables
    dataset['FamilySize'] = dataset ['SibSp'] + dataset['Parch'] + 1

    dataset['IsAlone'] = 1 #initialize to yes/1 is alone
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0 # now update to no/0 if family size is greater than 1

    #quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]


    #Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    #Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)

    #Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)


    
#cleanup rare title names
#print(data1['Title'].value_counts())
stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
title_names = (data1['Title'].value_counts() < stat_min) #this will create a true false series with title name as index

#apply and lambda functions are quick and dirty code to find and replace with fewer lines of code: https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/
data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
print(data1['Title'].value_counts())
print("-"*10)


#preview data again
data1.info()
data_val.info()
data1.sample(10)

In [ ]:
#draw a bar plot of survival by sex
fig, saxis = plt.subplots(2, 3,figsize=(16,12))
sns.barplot(x="Sex", y="Survived", data=train, ax =saxis[0,0])
sns.barplot(x="Pclass", y="Survived", data=train, ax =saxis[0,1])
sns.pointplot(x = 'Age', y = 'Survived',  data=train, ax = saxis[1,1])

In [ ]:
sns.barplot(x="Pclass", y="Survived", data=train)